In [10]:
# Install required packages
!pip install opencv-python
!pip install torch torchvision  # For PyTorch-based YOLO
!pip install tensorflow  # For TensorFlow-based YOLO (optional, if you prefer TensorFlow)
!pip install requests
!pip install python-dotenv

In [21]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [11]:
# Clone the YOLOv5 repository
!git clone https://github.com/ultralytics/yolov5.git
# Change the working directory to yolov5
%cd yolov5
# Install the required dependencies for YOLOv5
!pip install -r requirements.txt

Cloning into 'yolov5'...
remote: Enumerating objects: 17270, done.
remote: Counting objects: 100% (1/1), done.
remote: Total 17270 (delta 0), reused 0 (delta 0), pack-reused 17269 (from 2)
Receiving objects: 100% (17270/17270), 16.11 MiB | 15.30 MiB/s, done.
Resolving deltas: 100% (11861/11861), done.
/content/yolov5/yolov5


In [13]:
from google.colab import drive
drive.mount('/content/drive')
# Path to the image folder in Google Drive
image_folder = "/content/drive/MyDrive/IMG_for_object_detection/"

Mounted at /content/drive


In [15]:
import torch

# Load a pre-trained YOLOv5 model (can choose a larger model if needed)
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')  # 'yolov5s' is a smaller, faster version

Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2025-2-4 Python-3.11.11 torch-2.5.1+cu124 CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


In [19]:
# Perform inference on the first image
img = cv2.imread(image_folder + os.listdir(image_folder)[0])
results = model(img)

# Convert results to pandas DataFrame for the first image (index 0)
results_df = results.pandas().xywh[0]

# Print the first few rows to inspect the structure
print("Results DataFrame Columns:", results_df.columns)
print(results_df.head())

/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:894: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


Results DataFrame Columns: Index(['xcenter', 'ycenter', 'width', 'height', 'confidence', 'class', 'name'], dtype='object')
      xcenter     ycenter        width      height  confidence  class   name
0  933.199707  777.739746   106.689880  100.312317    0.487601     74  clock
1  520.576355  795.154541  1041.152710  537.538513    0.265898     59    bed
2   41.387009  531.456055    82.082291  190.181061    0.259947     56  chair


In [29]:
import cv2
import numpy as np
import os

# List all image files in the folder
image_files = [f for f in os.listdir(image_folder) if f.endswith(('.jpg', '.png', '.jpeg'))]

# Loop over each image in the folder
detection_results = []  # List to store results

# Iterate through all images in the folder
for image_file in image_files:
    img_path = os.path.join(image_folder, image_file)
    img = cv2.imread(img_path)

    # Perform inference on the image
    results = model(img)

    # Convert results to pandas DataFrame
    results_df = results.pandas().xywh[0]

    # Loop through the results and extract relevant information
    for index, row in results_df.iterrows():
        # Calculate xmin, ymin, xmax, ymax from the YOLO output
        xmin = row['xcenter'] - row['width'] / 2
        xmax = row['xcenter'] + row['width'] / 2
        ymin = row['ycenter'] - row['height'] / 2
        ymax = row['ycenter'] + row['height'] / 2

        detection_results.append({
            "image_name": image_file,
            "class": row['name'],  # Class name (e.g., 'clock')
            "confidence": row['confidence'],
            "xmin": xmin,
            "ymin": ymin,
            "xmax": xmax,
            "ymax": ymax
        })
print(detection_results[0])

{'image_name': 'CheMed123_64.jpg', 'class': 'clock', 'confidence': 0.4876008927822113, 'xmin': 879.8547668457031, 'ymin': 727.5835876464844, 'xmax': 986.5446472167969, 'ymax': 827.8959045410156}


In [ ]:
# To save the output to the database
from dotenv import load_dotenv
import psycopg2

# Load environment variables
load_dotenv()

# Database credentials
DB_NAME = os.getenv("POSTGRES_NAME")
DB_USER = os.getenv("POSTGRES_USER")
DB_PASSWORD = os.getenv("POSTGRES_PASSWORD")
DB_HOST = os.getenv("POSTGRES_HOST")
DB_PORT = os.getenv("POSTGRES_PORT")

# Establish a connection to the PostgreSQL database
try:
    conn = psycopg2.connect(host=DB_HOST, database=DB_NAME, user=DB_USER, password=DB_PASSWORD)
    cur = conn.cursor()

    # Create a table if it doesn't exist (adjust table and column names if needed)
    cur.execute("""
        CREATE TABLE IF NOT EXISTS object_detections (
            id SERIAL PRIMARY KEY,
            image_name VARCHAR(255),
            class VARCHAR(255),
            confidence FLOAT,
            xmin FLOAT,
            ymin FLOAT,
            xmax FLOAT,
            ymax FLOAT
        )
    """)
    conn.commit()

    # Insert the detection results into the database
    for detection in detection_results:
        cur.execute("""
            INSERT INTO object_detections (image_name, class, confidence, xmin, ymin, xmax, ymax)
            VALUES (%s, %s, %s, %s, %s, %s, %s)
        """, (detection['image_name'], detection['class'], detection['confidence'], detection['xmin'], detection['ymin'], detection['xmax'], detection['ymax']))
    conn.commit()

    print("Data successfully inserted into the PostgreSQL database.")

except psycopg2.Error as e:
    print(f"Error connecting to PostgreSQL or inserting data: {e}")

finally:
    if conn:
        cur.close()
        conn.close()
